In [3]:
from ipywidgets import HTML
import ipyvuetify as v

from sepal_ui import widgetFactory as wf
from sepal_ui import widgetBinding as wb
from sepal_ui.tiles import tiles, dateselector
from sepal_ui import aoi
from sepal_ui.styles.js import *

ResizeTrigger()

HTML(value='\n<style>\n.leaflet-pane {\n    z-index : 2 !important;\n}\n.leaflet-top, .leaflet-bottom {\n    z…

In [4]:
process_text = HTML("""<h1>Soil moisture mapping</h1>
</br>
<p>This module runs PYSMM, a python package for creating soil moisture maps. The module can be 
used to create a time series of soil moisture maps. The estimation of soil moisture is based on 
a Support-Vector-Regression machine learning approach, training the model based on in-situ 
data from the International Soil Moisture Network (ISMN) on cross-calibration between Sentinel-1 
and Global Land Data Assimilation System (GLDAS). More information about PYSMM can be found on 
<a href='https://pysmm.readthedocs.io/en/latest/'> the docs</a>.</p>
<p>To start the process, go through the steps 2 to 4.</p>
<br>
<p>*The PYSMM package was developed by Felix Greifeneder with modification from the SEPAL team.</p>

""")

In [5]:
date_text = HTML("""<h1>Date selector</h1>
</br>
<p>With the following menu you can select the date of the images that you want to run the algorithm, 
there are three options:</p>
<ul>
<li><b>All time series</b>: Will search all the avaiable images for the given area.</li>
<li><b>Single date</b>: Will search the closes image for the given date and area. </li>
<li><b>Range</b>: Will search all the images present in the given date range and area.</li>
</ul>
""")

In [6]:
start_text = HTML("""<h1>Soil Moisture Process</h1>
</br>
<p>After clicking in the 'Start' button, the availability of Sentinel-1 data is assessed and the command 
is sent through SEPAL to Google Earth Engine (GEE) to run the classification of soil moisture. This process could 
take a long time depending on the dimensions of the feature and on the number of the images to be processed.</p>

<p>If the selected dates are not available, the system will display a message with the closest images to the input dates.
The most recent image available depends on the GLDAS product, which has a delay of 1 to 2 months.</p>
<p>The green Processing bar shows the name of the task that is sent to GEE. When the processing reaches 100% all 
the tasks have been sent to GEE and the classification of soil moisture will continue there.</p>
<p>After all the tasks are sent to GEE the module can be closed. The processing will continue uninterrupted in GEE. 
In GEE the processing can take hours or days depending on the size of the AOI and the date range selected. """)

In [7]:
# The first process is diveded into 3 steps
# 1: Aoi selection
# 2: Date selection
# 3: Run process and display outputs

In [8]:
# Step1: Aoi selection
aoi_io = aoi.Aoi_IO(asset_id='users/dafguerrerom/ReducedAreas_107PHU')
aoi_tile = tiles.aoi_tile(aoi_io, remove_method=['Country boundaries', 'Draw a shape', 'Upload file'])

In [9]:
# 2: Date selection
date_obj = dateselector.DateSelector()
date_tile = dateselector.date_picker_tile(date_obj)

In [10]:
# 3: Run process and display outputs
run_tile = tiles.run_process_tile(aoi_io, date_obj)

In [11]:
step0_content = v.StepperContent(
    key=1,
    step=1, 
    children=[
        v.Card(
            children=[
                process_text,
            ]
        )
        
    ]
)  

step1_content = v.StepperContent(
    key=2,
    step=2, 
    children=[
        v.Card(
            children=[
                aoi_tile
            ]
        )
        
    ]
)  

step2_content = v.StepperContent(
key=3,
step=3, 
children=[
    v.Card(
        children=[
            date_text,
            date_tile
        ],
        class_="mb-12",
    ),
]
)

step3_content = v.StepperContent(
key=4,
step=4, 
children=[
    v.Card(
        children=[
            start_text,
            run_tile
        ],
        class_="mb-12 pa-5",
    ),
]
)

stepper_children = [
    v.StepperStep(
        key=1,
        complete=False,
        step=1,
        editable=True,
        children=[f"Introduction"]
    ),
    v.Divider(),
    v.StepperStep(
        key=2,
        complete=False,
        step=2,
        editable=True,
        children=[f"AOI selection"]
    ),
    v.Divider(),
    v.StepperStep(
        key=3,
        complete=False,
        step=3,
        editable=True,
        children=[f"Date selection"]
    ),
    v.Divider(),
    v.StepperStep(
        key=4,
        complete=False,
        step=4,
        editable=True,
        children=[f"Run proces"]
    ),
]

stepper_header = v.StepperHeader(
    children=stepper_children
)

aoi_content = v.Stepper(
         children=[
             stepper_header,
             v.StepperItems(
                 children=[
                     step0_content,
                     step1_content,
                     step2_content,
                     step3_content
                 ]
             )
         ]
)

In [12]:
def resize_map(widget, data, change):
    global rt
    rt.resize+=1
aoi_content.children[0].children[2].on_event('click', resize_map)

In [14]:
process_tile = wf.Tile('aoi_tile', 'Parameter selection', inputs=[aoi_content])

In [15]:
process_tile

Layout(align_center=True, children=[Card(children=[Html(children=['Parameter selection'], tag='h2'), Flex(chil…